#**setting**

##installs and keys

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


In [2]:
!pip install 'crewai[tools]'


  Using cached crewai_tools-0.1.7-py3-none-any.whl (47 kB)
  Attempting uninstall: crewai-tools
    Found existing installation: crewai-tools 0.1.6
    Uninstalling crewai-tools-0.1.6:
      Successfully uninstalled crewai-tools-0.1.6


In [3]:
# !pip install groq

# import os
# from groq import Groq

# # Set your Groq API key
# os.environ['GROQ_API_KEY'] = ''
# os.environ['GROQ_MODEL_NAME'] = 'llama3-8b-8192'

# # Initialize the Groq client
# groq_api_key = os.getenv('GROQ_API_KEY')
# model_name = os.getenv('GROQ_MODEL_NAME')
# client = Groq(api_key=groq_api_key)


In [4]:
# import os

# # Set your Serper API key
# os.environ['SERPER_API_KEY'] = ''


In [5]:
# Step 1: Install necessary packages
!pip install groq

# Step 2: Import necessary libraries
import os
from groq import Groq
from google.colab import userdata

# Step 3: Retrieve API keys from Colab secrets manager
groq_api_key = userdata.get('GROQ_API_KEY')
model_name = userdata.get('GROQ_MODEL_NAME')  # Assuming this is used later

# Step 4: Initialize the Groq client using the retrieved secrets
client = Groq(api_key=groq_api_key)

# Step 5: Verify the setup
print(f"Initialized Groq client with API key.")

# If the model_name is needed later, you can use it as follows:
print(f"Model name: {model_name}")

# Retrieve other API keys as needed
serper_api_key = userdata.get('SERPER_API_KEY')


Initialized Groq client with API key.
Model name: llama3-8b-8192


##crewAI tools import

In [6]:
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew, Process



In [7]:
from crewai_tools import SerperDevTool

# Initialize the Serper tool
serper_tool = SerperDevTool()


#crew

## Custom Agent and Custom Task Definitions

In [8]:
class CustomAgent:
    def __init__(self, role, goal, verbose=False, memory=False, backstory="", tools=None):
        self.role = role
        self.goal = goal
        self.verbose = verbose
        self.memory = memory
        self.backstory = backstory
        self.tools = tools if tools else []

    def perform_task(self, task_description):
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": f"Role: {self.role}, Goal: {self.goal}, Backstory: {self.backstory}"},
                {"role": "user", "content": task_description}
            ],
            model=model_name,
            temperature=0.8,
            max_tokens=2048,
            top_p=0,
        )
        return response.choices[0].message.content

class CustomTask:
    def __init__(self, description, expected_output, agent, async_execution=False, output_file=None):
        self.description = description
        self.expected_output = expected_output
        self.agent = agent
        self.async_execution = async_execution
        self.output_file = output_file

    def execute(self):
        result = self.agent.perform_task(self.description)
        if self.output_file:
            with open(self.output_file, 'w') as file:
                file.write(result)
        return result


## management

In [9]:
# Define the Account Executive Manager Agent with questioning functionality
account_executive_manager = CustomAgent(
    role='Account Executive Manager',
    goal='Assist the account manager and handle day-to-day client interactions',
    verbose=True,
    memory=True,
    backstory=(
        "You assist the account manager and handle day-to-day client interactions. "
        "Your goal is to ensure smooth communication and execution of client projects."
    )
)

# Define the task for the Account Executive Manager with direct questioning
account_executive_manager_task = CustomTask(
    description=(
        "Assist the account manager and handle day-to-day client interactions. "
        "Communicate with clients to gather feedback and ensure their needs are addressed. \n"
        "You have a spark of genius, that is why you are here. \n"
        "So write it within the depths of your heart and come up with insights on clients answers. \n"
        "Please answer the following questions to gather necessary information and come with your beautiful insights on it:\n"
        "1. What to advertise?\n"
        "2. When to advertise?\n"
        "3. Where does it sell?\n"
        "4. What's the budget?\n"
        "5. Add anything you want.\n"
        "remember to entitle your document and enlist the questions with their exact answeres at the beginning of your report.\n"
        "remember to end your document with your sign (pick your name yourself) and the new warmhearted joke upon the subject."
    ),
    expected_output='A client feedback report with answers to the questions and your insights.',
    agent=account_executive_manager,
    async_execution=False,
    output_file='1_account_executive_brief.txt'
)

# Simulate user interaction
def simulate_user_interaction(task_description):
    #display_with_delay(task_description)
    answers = {}
    answers['What to advertise?'] = input("What to advertise? ")
    answers['When to advertise?'] = input("When to advertise? ")
    answers['Where does it sell?'] = input("Where does it sell? ")
    answers['What is the budget?'] = input("What is the budget? ")
    answers['Add anything you want.'] = input("Add anything you want. ")
    return answers

# Execute the task with user interaction
def execute_task_with_user_interaction(task):
    answers = simulate_user_interaction(task.description)
    task.description += "\n".join([f"{question} {answer}" for question, answer in answers.items()])
    result = task.execute()
    return result


## strategy

In [10]:
# Define the Strategist Agent
strategist = CustomAgent(
    role='Business Strategist',
    goal='Challenge the document and develop a unique strategic plan based on the account brief and client feedback',
    verbose=True,
    memory=True,
    backstory=(
        "As a Business Strategist, you are responsible for crafting comprehensive strategic plans that align with client goals "
        "and feedback provided in the account brief. Your expertise helps clients achieve their business objectives."
    )
)

# Define the task for the Strategist
strategist_task = CustomTask(
    description=(
        "Challenge the given strategy, push out of it to the new hights, think outside the box and Develop a completely new strategic plan based on the data provided in the account brief and client feedback. "
        "Ensure the strategy addresses client goals, key insights, and specific requirements. "
        "Remember to entitle your document and list the questions with their exact answers at the beginning of your report.\n"
        "Remember to end your document with your sign, your name and title (not those that are already mentioned), and a new warmhearted joke upon the subject."
    ),
    expected_output='A detailed strategic plan document.',
    agent=strategist,
    async_execution=False,
    output_file='2_strategic_plan.txt'

)


## research

In [11]:
# Define the Researcher Agent with the Serper tool
researcher = CustomAgent(
    role='Researcher',
    goal='Augment the strategic plan with additional research from the web',
    verbose=True,
    memory=True,
    backstory=(
        "You are an expert in conducting thorough online research to gather relevant information "
        "that can enhance strategic plans. Review 3 competitors, provide lonks to ad campaigns reviews. Your goal is to ensure that the strategist's plan is "
        "well-informed and backed by comprehensive research."
    ),
    tools=[serper_tool]
)

# Define the task for the Researcher
researcher_task = CustomTask(
    description=(
        "Read the strategic plan and perform web searches to gather additional information. Review 3 competitors, provide lonks to ad campaigns reviews. "
        "Augment the strategic plan with this research and create a new, comprehensive report."
    ),
    expected_output='An augmented strategic plan document with additional research.',
    agent=researcher,
    async_execution=False,
    output_file='3_augmented_strategic_plan.txt'
)



## creative director

In [12]:
# Define the Creative Director Agent with the Serper tool
creative_director = CustomAgent(
    role='Creative Director',
    goal='Create a creative brief that includes consumer insights and references from social life and art',
    verbose=True,
    memory=True,
    backstory=(
        "You are responsible for crafting compelling creative briefs that provide valuable insights for creatives. "
        "Your expertise in finding relevant references from social life and art ensures that the creative team has "
        "the necessary inspiration and guidance to develop impactful content."
    ),
    tools=[serper_tool]
)

# Define the task for the Creative Director
creative_director_task = CustomTask(
    description=(
        "Read the augmented strategic plan and think waht should you search for online to make your research. Perform web searches to gather references for future advertising campaign from social life and art. "
        "Develop three meticulous consumer insights that will help a writer craft compelling copy. "
        "Create a comprehensive creative brief that includes these insights and the augmented strategic plan."
    ),
    expected_output='A creative brief document with consumer insights and references from social life and art.',
    agent=creative_director,
    async_execution=False,
    output_file='4_creative_brief.txt'
)


## copywriter

In [13]:
# Define the Copywriter Agent
copywriter = CustomAgent(
    role='Copywriter',
    goal='Craft compelling copy based on the creative brief',
    verbose=True,
    memory=True,
    backstory=(
        "You are an expert in crafting persuasive and engaging copy. "
        "Your role is to take the creative brief and develop copy that effectively communicates the message "
        "to the target audience."
    )
)

# Define the task for the Copywriter
copywriter_task = CustomTask(
    description=(
        "Read the creative brief and develop compelling copy that effectively communicates the message to the target audience. "
        "Ensure that the copy aligns with the consumer insights and references from social life and art provided in the brief."
    ),
    expected_output='A document containing the crafted copy.',
    agent=copywriter,
    async_execution=False,
    output_file='5_crafted_copy.txt'
)


## art director

In [14]:
# Define the Art Director Agent with the Serper tool
art_director = CustomAgent(
    role='Art Director',
    goal='Augment the crafted copy with art visions and ideas for visual materials',
    verbose=True,
    memory=True,
    backstory=(
        "You are a visionary with a keen eye for aesthetics. "
        "Your role is to enhance the crafted copy with art visions and ideas for visual materials. "
        "You will use web searches to find references on style and tricks that can inspire the creative approach."
    ),
    tools=[serper_tool]
)

# Define the task for the Art Director
art_director_task = CustomTask(
    description=(
        "Read the crafted copy and creative brief. Perform web searches to gather references on style and tricks. "
        "Augment the crafted copy with art visions and ideas for visual materials. "
        "Create a comprehensive document that outlines the creative approach."
    ),
    expected_output='A creative approach document with art visions and ideas for visual materials.',
    agent=art_director,
    async_execution=False,
    output_file='6_creative_approach.txt'
)


#run all

##Setting the Stage (Console Appearance)


In [15]:
import os
import time
import sys

def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

def print_green(text, **kwargs):  # Accept and pass through keyword arguments
    print("\033[92m{}\033[0m".format(text), **kwargs)

def display_with_delay(text, delay=0.02):
    for char in text:
        print_green(char, end='')
        sys.stdout.flush()
        time.sleep(delay)
    print('')  # Newline after the loop

## all execute comands under one main func

In [16]:
def run_agency():
    clear_screen()
    display_with_delay("=== ADVERTISING AIGENCY 22 ===")

    #ACCOUNT MANAGEMENTT

    # Execute the task for the Account Executive Manager
    result = execute_task_with_user_interaction(account_executive_manager_task)
    # Print the result
    print(result)


    #STRATEGY

    # Read the client feedback from the text file
    with open('1_account_executive_brief.txt', 'r') as file:
        client_data = file.readlines()

    # Convert client feedback to a task description
    strategist_task.description += "\nClient Feedback:\n"
    for line in client_data:
        strategist_task.description += line

    # Execute the task for the Strategist
    strategic_plan_result = strategist_task.execute()
    # Print the strategic plan result
    print(strategic_plan_result)


    #RESEARCH

    # Read the strategic plan from the text file
    with open('2_strategic_plan.txt', 'r') as file:
        strategic_plan_content = file.read()

    # Convert the strategic plan content to a task description
    researcher_task.description += "\nStrategic Plan:\n" + strategic_plan_content

    # Execute the task for the Researcher
    augmented_strategic_plan_result = researcher_task.execute()
    # Print the augmented strategic plan result
    print(augmented_strategic_plan_result)


    #CREATIVE DIRECTION

    # Read the augmented strategic plan from the text file
    with open('3_augmented_strategic_plan.txt', 'r') as file:
        augmented_strategic_plan_content = file.read()

    # Convert the augmented strategic plan content to a task description
    creative_director_task.description += "\nAugmented Strategic Plan:\n" + augmented_strategic_plan_content

    # Execute the task for the Creative Director
    creative_brief_result = creative_director_task.execute()
    # Print the creative brief result
    print(creative_brief_result)


    #COPYWRITING

    # Read the creative brief from the text file
    with open('4_creative_brief.txt', 'r') as file:
        creative_brief_content = file.read()

    # Convert the creative brief content to a task description
    copywriter_task.description += "\nCreative Brief:\n" + creative_brief_content

    # Execute the task for the Copywriter
    crafted_copy_result = copywriter_task.execute()
    # Print the crafted copy result
    print(crafted_copy_result)


    #ART DIRECTION

    # Read the crafted copy and creative brief from the text files
    with open('5_crafted_copy.txt', 'r') as file:
        crafted_copy_content = file.read()

    with open('4_creative_brief.txt', 'r') as file:
        creative_brief_content = file.read()

    # Convert the crafted copy and creative brief content to a task description
    art_director_task.description += "\nCrafted Copy:\n" + crafted_copy_content
    art_director_task.description += "\nCreative Brief:\n" + creative_brief_content

    # Execute the task for the Art Director
    creative_approach_result = art_director_task.execute()
    # Print the creative approach result
    print(creative_approach_result)

##run main func

In [ ]:
if __name__ == "__main__":
    run_agency()

=== ADVERTISING AIGENCY 22 ===
